In [48]:
import pandas as pd
dfFivePoints = pd.read_csv('Data/restructured_fivepointstest.csv')

In [49]:
rows = dfFivePoints['student_id'].value_counts().reset_index()
rows.columns = ['student_id', 'total_patterns']

# Groepeer de rijen op basis van 'student_id' en neem de som van de 'pattern' waar deze gelijk is aan "0,0,0,0,0,0,0,0"
sum_of_pattern = dfFivePoints[dfFivePoints['patternsm'] == "0,0,0,0,0,0,0,0"].groupby('student_id')['patternsm'].count().reset_index()
sum_of_pattern.columns = ['student_id', 'sum_invalid_pattern']

total_duplicates = dfFivePoints.groupby('student_id').size().reset_index()
total_duplicates.columns = ['student_id', 'total_duplicates']

# Voeg de sommen toe aan de oorspronkelijke DataFrame
rows = pd.merge(rows, sum_of_pattern, on='student_id', how='left')

rows

,student_id,total_patterns,sum_invalid_pattern
0,879,209,192.0
1,1161,175,156.0
2,748,164,137.0
3,9884,151,66.0
4,2639,137,77.0
...,...,...,...
4545,1354,1,1.0
4546,9780,1,1.0
4547,7132,1,1.0
4548,5300,1,1.0


In [50]:
# Filter out rows with 'patternsm' not equal to '0,0,0,0,0,0,0,0'
non_zero_df = dfFivePoints[dfFivePoints['patternsm'] != '0,0,0,0,0,0,0,0']

# Group by 'student_id' and count the unique patterns
unique_patterns_per_student = non_zero_df.groupby('student_id')['patternsm'].nunique().reset_index()
unique_patterns_per_student.columns = ['student_id', 'unique_patterns_count']

# Calculate the total count of values for each student, including zeros
total_values_per_student = dfFivePoints.groupby('student_id')['patternsm'].count().reset_index()
total_values_per_student.columns = ['student_id', 'total_values_count']

# Calculate the total count of values for each student, excluding zeros
total_non_zero_values_per_student = non_zero_df.groupby('student_id')['patternsm'].count().reset_index()
total_non_zero_values_per_student.columns = ['student_id', 'total_non_zero_values_count']

# Merge the DataFrames to create the new DataFrame
new_df = pd.merge(unique_patterns_per_student, total_values_per_student, on='student_id')
new_df = pd.merge(new_df, total_non_zero_values_per_student, on='student_id')

# Calculate the "duplicates" column
new_df['duplicates'] = new_df['total_non_zero_values_count'] - new_df['unique_patterns_count']

new_df.head()

,student_id,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
0,1,5,12,6,1
1,2,26,28,28,2
2,3,38,48,47,9
3,4,54,64,62,8
4,5,48,63,62,14


In [51]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4535 entries, 0 to 4534
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   student_id                   4535 non-null   int64
 1   unique_patterns_count        4535 non-null   int64
 2   total_values_count           4535 non-null   int64
 3   total_non_zero_values_count  4535 non-null   int64
 4   duplicates                   4535 non-null   int64
dtypes: int64(5)
memory usage: 177.3 KB


In [52]:
# combine the DataFrames
combine = pd.merge(rows, new_df, on='student_id', how='left')

combine.head()

,student_id,total_patterns,sum_invalid_pattern,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
0,879,209,192.0,6.0,209.0,17.0,11.0
1,1161,175,156.0,17.0,175.0,19.0,2.0
2,748,164,137.0,25.0,164.0,27.0,2.0
3,9884,151,66.0,28.0,150.0,84.0,56.0
4,2639,137,77.0,16.0,137.0,60.0,44.0


In [53]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4550 entries, 0 to 4549
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   student_id                   4550 non-null   int64  
 1   total_patterns               4550 non-null   int64  
 2   sum_invalid_pattern          2709 non-null   float64
 3   unique_patterns_count        4535 non-null   float64
 4   total_values_count           4535 non-null   float64
 5   total_non_zero_values_count  4535 non-null   float64
 6   duplicates                   4535 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 249.0 KB


In [54]:
#check student_id 1 in df combine
combine[combine['student_id'] == 1]

,student_id,total_patterns,sum_invalid_pattern,unique_patterns_count,total_values_count,total_non_zero_values_count,duplicates
4435,1,12,6.0,5.0,12.0,6.0,1.0


In [55]:
# make a cvs file for the combine DataFrame
combine.to_csv('Data/TotalscoreAndDuplicated.csv', index=False)